# Gravity of Probability – Boundary Diagnostics

Quickstart notebook for stacking CMB temperatures around DESI voids and inspecting
large-angle monopole/dipole behavior.

This notebook is exploratory and diagnostic only..
It does NOT tune or modify any Gravity of Probability core-parameters.

In [ ]:
# import
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

from gop_boundary_desi.io import read_void_catalog
from gop_boundary_desi.stacking import stack_cmb_temperature_profile
from gop_boundary_desi.dipole import fit_monopole_dipole


In [ ]:
# Update paths to your local data
void_catalog_path = "../data/desi_voids.fits"
cmb_map_path = "../data/planck_smica.fits"
mask_path = "../data/mask.fits"  # optional

voids = read_void_catalog(void_catalog_path)
cmb_map = hp.read_map(cmb_map_path, field=0, verbose=False)
nside = hp.get_nside(cmb_map)

mask = None
try:
    mask_map = hp.read_map(mask_path, field=0, verbose=False)
    mask = mask_map != 0
except FileNotFoundError:
    print("No mask found - proceeding without mask")


In [ ]:
# void stacking
theta_max_deg = 5.0
nbins = 15
theta_bins = np.linspace(0.0, theta_max_deg, nbins + 1)

result = stack_cmb_temperature_profile(
    voids=voids,
    cmb_map=cmb_map,
    nside=nside,
    theta_bins_deg=theta_bins,
    mask=mask,
)

print(f"Used {result.n_used} voids")


In [ ]:
# plot
plt.errorbar(
    result.theta_bins_deg,
    result.prof_mean_uK,
    yerr=result.prof_sem_uK,
    fmt="o",
    capsize=3,
)

plt.axhline(0, color="k", lw=0.8)
plt.xlabel("Angular separation from void center (deg)")
plt.ylabel("Mean CMB temperature (µK)")
plt.title("Void-stacked CMB temperature profile")
plt.show()


In [ ]:
# dipole diagnostic
dfit = fit_monopole_dipole(cmb_map, mask=mask)

print("Monopole + dipole fit")
print("---------------------")
print(f"Dipole amplitude : {dfit.amp:.4e}")
print(f"Dipole axis (x,y,z): {dfit.axis_vec}")
print(f"chi2 / dof       : {dfit.chi2 / dfit.dof:.4e}")


In [ ]:
## Interpretation Notes

- This notebook tests whether large-angle residuals or void-stacked temperature features are present in the data.
- No Gravity of Probability parameters are adjusted here. The analysis is purely diagnostic.
- Any observed dipole or boundary-aligned feature should be interpreted as a candidate geometric/boundary imprint,
  consistent with decoherence-boundary fossilization, not as a detection claim.
- Null results are scientifically meaningful and expected in some regimes.
